# Sick Slopes
### Finding routes maximize speed in unpowered personal transport

The hobby I've spent the most time doing is [longboarding](https://en.wikipedia.org/wiki/Longboarding), where my board is basically a large skateboard optimized for cruising around. I have spent many days and nights cruising in Atlanta, Houston, Dallas, and Barcelona. Often, I want to find places where I can fly downhill fast, especially in routes uninterrupted by stop signs, stoplights, and other barriers where I can slow down before encountering anything interesting. Basically, I just want to find the routes where I can go fast.

## Existing Soutions
* [Findhills](http://www.findhills.com/) is the state-of-the-art hillfinding technology. In the site, users can link together points on Google Maps to create a route that follows roads. In a sidebar, a gorgeous, interactive, color-coded graph shows steepness and elevation change
    * This tool is best used in conjunction with a topo map such as MyTopo or Google Maps terrain view
* Browsing the web
    * There's several YouTube Videos that reveal locations. Searching for specific cites can yeild surprisingly good results.
    * A couple of people have started spot databases. Check out [BoardSpots](http://boardspots.com/) and [Longboard Mapp](https://play.google.com/store/apps/details?id=com.adrenalynmapp.longboard&hl=en).
    * Longboarding forums like Siverfish (rip) and /r/longboarding are very stingy about posting spots since longboarding is illegal in so many places.
    
## Data Sources: OpenStreetMap

Since Google Maps doesn't allow for direct download of street data, OpenStreetMap (OSM) is the source from which we'll get street locations. OSM's API takes 2 points that represent a bounding box (rectangle of coordinates) and returns a list of elements, of which OSM has defined 3 types, all of which can have tags (attributes):
* Nodes, which are points with a specific corrdinate pair
* Ways, which are made up of points, typically roads, rivers or bounderies. The tag prefix 'highway' indicates any type of road or path.
* Relations, which are made up of nodes, ways, and other relations.

For this project we process this data by creating an adjacency list of all nodes found in ways with desirable attributes, all of which were originally defined in British terms. We eliminate roads marked as 'highway=motorway' (ie. limited access freeway), highway=trunk, and highway=primary. Some tags such as 'highway=steps' can definately be eliminated. Although the current implementation eliminates ways with 'surface=unpaved' and similar ways, this should be an editable attribute so bikers can enable unpaved surfaces and longboarders can disable them. We also eliminate bridges, since USGS typically provides elevation data for land under bridges rather than the bridge itself. With further testing, it might be advantagious to assume all bridges are flat. For one-way streets, we only complete the adjacency list for the legal direction.

We also generate from OSM a list of stop signs and traffic signals. Stop signs (and other equivilent markers) are unfortuately marked poorly by OSM: While there are over 1,000,000 signals marked, there are less than 400,000 stop signs marked, and I often see areas with no stop signs marked. While this may be because of the difficulty seeing stop signs from airial pictures, it might also be due to the policy on the marker for stopsigns, 'highway=stop', for interections where only some of the roads have stop signs (not an all way stop):

> Since the stop line on the approach applies to only one travel direction, that direction can usually be deduced by finding the shorter distance to the priority intersection. However, a few stop signs are on 2-way streets between closely-spaced junctions, making it necessary to identify the travel direction that stops. Where needed, this can be done using direction=forward or direction=backward, relating stop direction to OSM forward/backward of the way that contains the highway=stop node.

This is quite annoying for routing software that wants to account for stop signs. It shouldn't cause too many issues with my program though.

## Data Sources: USGS Elevation Data

USGS publishes elevation data in its National Elevation Dataset (NED). While the entire US besides Alaska has been mapped at the resolution of 1/3 arc second, some of the US based on what I assume is state and municipal funding has been mapped at resolutions of 1/9 arc second and 1 meter.

So how large is 1/3 of an arc second? The question is complicated if you're trying to deal with the earth's shape of an elipsoid with varying elevation, but we can approximate it as a sphere for our purposes since we're not trying to point satellites or anything. Lines of latitude 1/3 arc seconds apart are about 10.29 meters apart, as as are lines of longitude at the equator. However, lines of longitude become closer together farther away from the equator: 10.29 * cos(latitude) can calculate the distance between 2 lines of longitude 1/3 arc seconds apart.

In [2]:
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting
deg = np.arange(0.0, 90.01, 0.01)

def deg2dist(deg): return 10.29 * np.cos(np.pi / 180 * deg)
dist = deg2dist(deg)

# Note that using plt.subplots below is equivalent to using
# fig = plt.figure and then ax = fig.add_subplot(111)
fig, ax = plt.subplots()
ax.plot(deg, dist)

ax.set(xlabel='degrees', ylabel='Distance',
       title='Distance between lines of longitude 1/3 arc second apart')
ax.grid()

fig.savefig("degrees_and_dist.png")
plt.show()

print("Notable Latitudes")
print("49th parrallel (US-Canada border): ", deg2dist(49))
print("25.9 deg N (Brownsville, TX): ", deg2dist(25.9))
print("35.0 deg (LA): ", deg2dist(35.0))
print("40.7 deg (NYC): ", deg2dist(40.7))

Notable Latitudes
49th parrallel (US-Canada border):  6.75084740831
25.9 deg N (Brownsville, TX):  9.25644954545
35.0 deg (LA):  8.42907453573
40.7 deg (NYC):  7.80120231947


So how should we query points? 

I think I've given this problem a bit more thought than it deserves for my purposes, but I'll type it out here. I'm not sure how the USGS decides what single elevation should represent an area of land, but if I were them, I would either go for the the median elevation that appears to represent land in their point cloud, or I would try and represent a point towards the middle of the block. In either case, most slopes in life are rather continious, so the two should be simmilar

## Defining the problem

To calculate the acceleration of of a wheeled object on an incline (or lack thereof), we must consider its mass and the forces acting upon it, namely, the acceleration due to gravity, the wheel's internal bearing friction, the rolling friction, and air resistance. The former 3 contribute forces porportional to the sine of the incline; the latter is the daily work of thousands of engineers.

## Solving the problem

I've thought a lot about this problem, first talking about doing DFS and BFS based approaches from points of local maxima (and programming it), then talking about doing Djikstra-inspired algorithms with steeper slopes or fastest route expansions first. After taking Algorithms, I realize that the key to solving this is to minimize duplicate work! Luckily, we can do that. Hear me out.

* We can label each node with the speed at which we can start _speed_, typically 1 m/s for longboards, then set to previous node _prev_ to **None**.

* Consider the highest elevation node on our map. It would make sense for routes to start here, since all routes to this point are uphill. For each adjacent node, we can the node's _prev_ and _speed_ with the new highest speed if it is indeed higher.
    
* In fact, for each next highest point, we know that there cannot exist a faster way to get to the point, so we can expand its adjacent nodes too.

And that's it! We can construct the fastest route by starting at the highest speed node. The algorithm is O(nlogn + e) since we have to sort the nodes by elevation.

# Implementing

Let's start by downloading elevation data for a query. We query by using a simple dict.

In [32]:
data = {
    'north': 33.7874, 
    'west':  -84.4203, 
    'south': 33.7677,
    'east':  -84.3812, 
}

In [27]:
import wget
from scan_product_links import urls
import math
import os
us_urls = urls("elevationproductslinks/13secondplots.csv")
mx_ca_urls = urls("elevationproductslinks/1secondplots.csv")
def download_coords(data, country='United States'):
    # TODO check if request is gucci
    # TODO Remove the following block of code in production
    if country == 'United States':
        path_suffix = '_13'
        useful_urls = us_urls
    else:
        path_suffix = '_1'
        useful_urls = mx_ca_urls
    for lat in range(
        math.ceil(float(data['south'])), math.ceil(float(data['north'])) + 1
        # Eg N 87.7 to N 86.
        ):
        for lng in range(
            math.floor(float(data['west'])), math.floor(float(data['east'])) + 1
            ):
            fname = ('grd' + ('n' if lat>0 else 's')
                + str(abs(math.ceil(lat))).zfill(2)
                + ('e' if lng>=0 else 'w')
                + str(abs(math.floor(lng))).zfill(3))
            database_path = ('elevationdata/'
                + fname
                + path_suffix + '/w001001.adf'
                )
            if not os.path.exists(database_path):
                try:
                    print("downloading" + useful_urls[(lat, lng)] + "\n")
                    wget.download(useful_urls[(lat, lng)])
                    print("\n")
                    file_name = useful_urls[(lat, lng)].split('/')[-1]
                    archive = zipfile.ZipFile(file_name)
                    for file in archive.namelist():
                        if file.startswith("grd" + fname[3:] + path_suffix + "/"):
                            archive.extract(file, "elevationdata")
                    os.remove(file_name)
                except (urllib.error.HTTPError):
                    print("Could not download data for", (lat, lng))
                except KeyError:
                    print("Thing not found in urls: " (lat, lng))

In [28]:
download_coords(data)

In [34]:
mapsize = (
            data['west'],
            data['south'],
            data['east'],
            data['north']
            )

Next, we query OpenStreetMap for the map data!

In [30]:
import subprocess
import math
import osmapi
import os.path
import pickle
def get_map_data(mapsize):
    mapfilepath = 'maps/map'+str(mapsize)+'.dat'
    # TODO Allow spanning countries
                        # (west, south, east, north), string

    api_link = osmapi.OsmApi(#username='evanxq1@gmail.com',
                             #password='hrVQ*DO9aD9q'#,
                             #api="api06.dev.openstreetmap.org"
                             )
    try:
        if os.path.exists(mapfilepath):
            print('loading local map...')
            with open(mapfilepath, 'rb') as f:
                map_data = pickle.load(f)
        else:
            print('requesting map...')
            map_data = api_link.Map(mapsize[0], mapsize[1],
                                    mapsize[2], mapsize[3])
            with open(mapfilepath, 'wb') as f:
                pickle.dump(map_data, f) # TODO delete this entire try block tbh

    except IOError as e:
        print("Couldn't write map data!", e.errorno, e.strerror)

#     except Error as e: # osmapi.OsmApi.MaximumRetryLimitReachedError:  #TODO: handle errors
#         print(e.errorno, e.strerror)
#         print("Could not get map data!")
#         return False, [], [], [], [], [], []
    return map_data

In [35]:
get_map_data(mapsize)

requesting map...


[{'data': {'changeset': 21752899,
   'id': 69097091,
   'lat': 33.765318,
   'lon': -84.391904,
   'tag': {'highway': 'traffic_signals'},
   'timestamp': datetime.datetime(2014, 4, 17, 16, 56, 57),
   'uid': 1946151,
   'user': 'Jack the Ripper',
   'version': 4,
   'visible': True},
  'type': 'node'},
 {'data': {'changeset': 12331089,
   'id': 69097092,
   'lat': 33.765317,
   'lon': -84.393102,
   'tag': {},
   'timestamp': datetime.datetime(2012, 7, 19, 14, 23, 22),
   'uid': 330773,
   'user': 'Roadrunner21',
   'version': 3,
   'visible': True},
  'type': 'node'},
 {'data': {'changeset': 12331089,
   'id': 69097094,
   'lat': 33.76533,
   'lon': -84.3945719,
   'tag': {},
   'timestamp': datetime.datetime(2012, 7, 19, 14, 23, 22),
   'uid': 330773,
   'user': 'Roadrunner21',
   'version': 3,
   'visible': True},
  'type': 'node'},
 {'data': {'changeset': 3110547,
   'id': 69097099,
   'lat': 33.765343,
   'lon': -84.396128,
   'tag': {},
   'timestamp': datetime.datetime(2009, 11,

In [36]:
map_data = get_map_data(mapsize)

loading local map...


We need to turn the map into a usable graph structure now!

Possible optimization: right now I have each node store its neighbors in IDs and as pointers. Since pointer dereferencing is slow, what we really want to do is to assign a zero-indexed ID to each node so it can be referenced in a list.

In [104]:
class Node:
    def __init__(self, node_id, lat, lng, is_stoplight, adj):
        self.node_id = node_id
        self.lat = lat
        self.lng = lng
        self.is_stoplight = is_stoplight
        self.adj = adj
        self.edge_coords = None
        self.edge_elevations = []
    def create_adj_node_ptrs(self):
        self.adj_node_ptrs = list(nodes[adj_node_id] for adj_node_id in self.adj)

In [187]:
from collections import defaultdict
def map_to_graph(map_data):
    graph = defaultdict(set)
    for entry in map_data:
        if (entry['type'] == 'way'
                and 'data' in entry.keys()
                and 'tag' in entry['data'].keys()
                and 'highway' in entry['data']['tag'].keys()
                and entry['data']['tag']['highway'] != 'steps'
                and 'bridge' not in entry['data']['tag']
        ):
            road_nodes = entry['data']['nd']
            for i in range(len(road_nodes) - 1):
                graph[road_nodes[i]].add(road_nodes[i+1])
                graph[road_nodes[i+1]].add(road_nodes[i])
    return graph

In [188]:
adj_list = map_to_graph(map_data)

In [189]:
def get_node_entries(target_nodes, map_data):
    for item in map_data:
        item_id = item["data"]["id"]
        if item_id in target_nodes:
            yield (item_id, item)

In [190]:

node_lat_lng = []
datapts_per_degree = 10800
def create_node_list_with_elevations(adj_list, map_data):
    nodes = dict()
    node_heights, node_latlons = dict(), dict()
    stoplights = set()
    for node_id, node_info in get_node_entries(adj_list.keys(), map_data):
        nodes[node_id] = Node(
            node_id,
            lat=float(node_info['data']['lat']), 
            lng=float(node_info['data']['lon']),
            is_stoplight = 
                ('tag' in node_info['data'] and 'highway' in node_info['data']['tag']
                    and node_info['data']['tag']['highway'] == 'traffic_signals'
                ),
            adj = list(adj_list[node_id])
        )
    return nodes

In [191]:
nodes = create_node_list_with_elevations(adj_list, map_data)

In [192]:
for node_id, node in nodes.items():
    node.create_adj_node_ptrs()

In [193]:
import numpy as np
datapts_per_degree = 10800
def add_edges_return_queries(nodes):
    large_query = set()
    for node_id, node in nodes.items():
        edge_coords = []
        for adj_node in node.adj_node_ptrs:
            # Degrees aren't squares, so this isn't super valid, but it's not important.
            dist_in_degs = np.sqrt((node.lat - adj_node.lat)**2 + (node.lng - adj_node.lng)**2)
            n_steps = max(int(dist_in_degs * datapts_per_degree), 2)
            lat_steps = np.linspace(node.lat, adj_node.lat, num=n_steps, endpoint=True)
            lng_steps = np.linspace(node.lng, adj_node.lng, num=n_steps, endpoint=True)
            coords = list(zip(lat_steps, lng_steps))
            edge_coords.append(coords)
            large_query.update(coords)
        node.edge_coords = edge_coords
    return large_query


In [194]:
large_query = add_edges_return_queries(nodes)

In [195]:
# Make sure each node's edges start with the same coordinates
def test_edge_coords_start(nodes):
    for node_id, node in nodes.items():
        first = (node.lat, node.lng)
        for edge in node.edge_coords:
            assert edge[0] == first
test_edge_coords_start(nodes)
# Make sure each node's edges end with the same coordinates 
# as its adjacent node's start with
def test_edge_coords_end(nodes):
    for node_id, node in nodes.items():
        assert len(node.edge_coords) == len(node.adj) == len(node.adj_node_ptrs)
        for i in range(len(node.adj_node_ptrs)):
            assert (node.adj_node_ptrs[i].lat, node.adj_node_ptrs[i].lng) == node.edge_coords[i][-1]
test_edge_coords_end(nodes)

Possible optimization here: instead of +=ing a bunch of strings, we could use a stringbuilder type object

In [196]:
from collections import defaultdict
def build_query_text(large_query, country="United States"):
    queries = defaultdict(str)
    latlng_order = defaultdict(list)
    for lat_lng in large_query:
        lat, lng = lat_lng
        fname = ('grd' + ('n' if lat>0 else 's')
                 + str(abs(math.ceil(lat))).zfill(2)
                 + ('e' if lng>=0 else 'w') # lng = 0 block is all east I guess
                 + str(abs(math.floor(lng))).zfill(3)
                 )

        s = str(lng) + ' ' + str(lat) + '\n'
        queries[fname] += s
        latlng_order[fname].append(lat_lng)
    return queries, latlng_order

def query_elevations(queries, latlng_order, country="United States"):
    points = []
    elevations = []
    for fname in queries.keys():
        if country == 'United States': # TODO deal with AK
            database_path = 'elevationdata/' + fname + '_13/w001001.adf'
        if country == 'Mexico' or country == 'Canada' or country == None:
            # TODO deal with country == None which would be sorta weird
            database_path = 'elevationdata/' + fname + '_1/w001001.adf'
        proc = subprocess.Popen(
            ['gdallocationinfo', database_path, '-valonly', '-geoloc'],
            stdin=subprocess.PIPE, stdout=subprocess.PIPE,
            universal_newlines=True
        )
        output, err = proc.communicate(queries[fname])
        elevations += [float(s) for s in output.splitlines()]
        points += latlng_order[fname]
    if len(points) != len(elevations):
        raise Exception("Error querying points: " + str(len(points)) + " points, " + str(len(elevations)) + " elevations")
    ret = dict()
    for i in range(len(points)):
        ret[points[i]] = elevations[i]
    return ret

In [197]:
queries, latlng_order = build_query_text(large_query, country="United States")

In [198]:
elevations = query_elevations(queries, latlng_order, country="United States")

In [199]:
def set_node_elevations(nodes, elevations):
    for node_id, node in nodes.items():
        for edge in node.edge_coords:
            elevation_list = []
            for coord_pair in edge:
                elevation_list.append(elevations[coord_pair])
            node.edge_elevations.append(elevation_list)

In [200]:
set_node_elevations(nodes, elevations)

How nice! Now we need to decide what kind of edges we need to split on due to elevation differences. Let's test some stuff out.

In [204]:
deviations = []
high_deviations = []
low_deviations = []
for node_id, node in nodes.items():
    for i, edge in enumerate(node.edge_elevations):
        bottom_end = min(edge[0], edge[-1])
        top_end = max(edge[0], edge[-1])
        top = max(edge)
        bottom = min(edge)
        deviations.append(max(top-top_end, bottom_end-bottom))
        high_deviations.append(top-top_end)
        low_deviations.append(bottom_end-bottom)

In [206]:
print(0, len(deviations))
print(.01, sum([i >= .01 for i in deviations]))
print(.1, sum([i >= .1 for i in deviations]))
print(.3, sum([i >= .3 for i in deviations]))
print(1, sum([i >= 1 for i in deviations]))
print(3, sum([i >= 3 for i in deviations]))
print(5, sum([i >= 5 for i in deviations]))

0 28130
0.01 2226
0.1 1594
0.3 870
1 254
3 38
5 10


Note that all values are even since our graph is directed, so there are edges going in each direction. Here's a question: since we've been storing all the adjacent edges of each graph as a random ordered list, what is the expected runtime of finding the opposite edge to split if we split over 1 meter?

In [208]:
edges_to_scan = 0
for node_id, node in nodes.items():
    for i, edge in enumerate(node.edge_elevations):
        bottom_end = min(edge[0], edge[-1])
        top_end = max(edge[0], edge[-1])
        top = max(edge)
        bottom = min(edge)
        edges_to_scan += len(node.edge_elevations)
print(edges_to_scan/4) # over 4 rather than 2 since we only have to look at "opposite" nodes from the ones we split

18393.0


I've seen better, but it's not worth doing anything about now. Road graphs are always kind of sparse, so I imagine this would increase somewhat linearly with edge total. Programming splitting sounds like a pain, though. I'll do it later (tm)

In [211]:
# Todo add to node ititialziation
data['init_speed'] = 1.0
data['use_stoplights'] = True
for node_id, node in nodes.items():
    node.speed = data['init_speed']
    node.last_node = None
    node.path_start = node
    # TODO add to some other method
    node.elevation = node.edge_elevations[0][0]

Here comes that big dramatic |V|log|V| sort that all CS classes have been prepraring me for!!

In [213]:
sorted_nodes = sorted(nodes.values(), key=lambda n: -n.elevation)

Well that was quick. Let's get to the «physics»